<a href="https://colab.research.google.com/github/ValentinCord/HandOnAI_1/blob/main/Fire_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ **I**. Lecture des données depuis le cluster

✅ **II**. Fusion des données avec suppression des fichiers copiés

✅ **III**. Sauvegarde des données sur google.drive

✅ **IV**. Suppression des données abérrantes 

✅ **V**. Ajout de données utiles

✅ **VI**. Suppresion des doublons

✅ **VII**. Description des données

# **I. Reading database**

In [ ]:
import os
bases_path_after="bases"
if os.path.exists(bases_path_after) == False:
    os.makedirs(bases_path_after)

In [ ]:
!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_1.tar
!tar xf FIRE_DATABASE_1.tar -C 'bases' --one-top-level
!rm FIRE_DATABASE_1.tar

!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_2.tar
!tar xf FIRE_DATABASE_2.tar -C 'bases' --one-top-level
!rm FIRE_DATABASE_2.tar

!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_3.tar
!tar xf FIRE_DATABASE_3.tar -C 'bases' --one-top-level
!rm FIRE_DATABASE_3.tar

!wget https://download.smartappli.eu/big.tar
!tar xf big.tar -C 'bases' --one-top-level
!rm big.tar

!wget https://download.smartappli.eu/medium.tar
!tar xf medium.tar -C 'bases' --one-top-level
!rm medium.tar

!wget https://download.smartappli.eu/small.tar
!tar xf small.tar -C 'bases' --one-top-level
!rm small.tar

# **II. Merge data**

In [ ]:
!rm -r db
!mkdir -p db/fire db/start_fire db/no_fire

rm: cannot remove 'db': No such file or directory


In [ ]:
!find bases/FIRE_DATABASE_1/fire ! -name '*copie*' -exec cp "{}" db/fire \;
!find bases/FIRE_DATABASE_2/fire ! -name '*copie*' -exec cp "{}" db/fire \;
!find bases/FIRE_DATABASE_3/fire ! -name '*copie*' -exec cp "{}" db/fire \;
!find bases/big/fire ! -name '*copie*' -exec cp "{}" db/fire \;
!find bases/medium/fire ! -name '*copie*' -exec cp "{}" db/fire \;
!find bases/small/fire ! -name '*copie*' -exec cp "{}" db/fire \;

!find bases/FIRE_DATABASE_1/start_fire ! -name '*copie*' -exec cp "{}" db/start_fire \;
!find bases/FIRE_DATABASE_2/start_fire ! -name '*copie*' -exec cp "{}" db/start_fire \;
!find bases/FIRE_DATABASE_3/start_fire ! -name '*copie*' -exec cp "{}" db/start_fire \;
!find bases/big/start_fire ! -name '*copie*' -exec cp "{}" db/start_fire \;
!find bases/medium/start_fire ! -name '*copie*' -exec cp "{}" db/start_fire \;
!find bases/small/start_fire ! -name '*copie*' -exec cp "{}" db/start_fire \;

!find bases/FIRE_DATABASE_1/no_fire ! -name '*copie*' -exec cp "{}" db/no_fire \;
!find bases/FIRE_DATABASE_2/no_fire ! -name '*copie*' -exec cp "{}" db/no_fire \;
!find bases/FIRE_DATABASE_3/no_fire ! -name '*copie*' -exec cp "{}" db/no_fire \;
!find bases/big/no_fire ! -name '*copie*' -exec cp "{}" db/no_fire \;
!find bases/medium/no_fire ! -name '*copie*' -exec cp "{}" db/no_fire \;
!find bases/small/no_fire ! -name '*copie*' -exec cp "{}" db/no_fire \;

# **III. Store database on google.drive**



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -av '/content/db/' '/content/gdrive/MyDrive/HandOnAI_Full_Fire_Classification/'

In [ ]:
drive_data_folder = '/content/gdrive/MyDrive/HandOnAI_Full_Fire_Classification/'

# **IV. Delete wrong data**

In [ ]:
img_aberrant = {
    'fire': ['00000335.jpg',
             'images - 2022-05-04T104033.252.jpeg',
             'images - 2022-05-04T104034.807.jpeg',
             'images - 2022-05-04T104044.688.jpeg',
             'images - 2022-05-04T104043.935.jpeg',
             'images - 2022-05-04T104047.349.jpeg'],
    'start_fire': [],
    'no_fire': []
    }

for classe, images in img_aberrant.items():
  
  for image in images:
    
    path = drive_data_folder + classe + '/' + image
    
    if os.path.exists(path):
      os.remove(path)

    else:
      print(f'{path} not found')


# **V. Adding data**

In [ ]:
!pip install git+https://github.com/Joeclinton1/google-images-download.git

from google_images_download import google_images_download

# instantiate the class
response = google_images_download.googleimagesdownload()
arguments = {"keywords": "forest, red panda in a forest, sunset in a forest, fall in forest",
             "limit": 50, "print_urls": False}
paths = response.download(arguments)

!cp -av '/content/downloads/ fall in forest/.' '/content/gdrive/MyDrive/HandOnAI_Full_Fire_Classification/no_fire/'
!cp -av '/content/downloads/ red panda in a forest/.' '/content/gdrive/MyDrive/HandOnAI_Full_Fire_Classification/no_fire/'
!cp -av '/content/downloads/ sunset in a forest/.' '/content/gdrive/MyDrive/HandOnAI_Full_Fire_Classification/no_fire/'
!cp -av '/content/downloads/forest/.' '/content/gdrive/MyDrive/HandOnAI_Full_Fire_Classification/no_fire/'

!rm -r downloads

# instantiate the class
response = google_images_download.googleimagesdownload()
arguments = {"keywords": "fire forest",
             "limit": 50, "print_urls": False}
paths = response.download(arguments)

!cp -av '/content/downloads/fire forest/.' '/content/gdrive/MyDrive/HandOnAI_Full_Fire_Classification/fire'
!rm -r downloads


# **VI. Delete duplicate data**

In [ ]:
!pip install ImageHash

In [ ]:
import imagehash
from tqdm import tqdm
from PIL import Image

In [ ]:
from traitlets.traitlets import default
classes = ['fire/', 'start_fire/', 'no_fire/']

hash_dict = {}

for classe in classes:

  path = drive_data_folder + classe

  for image in tqdm(os.listdir(path)):

    path_image = path + image

    hash = imagehash.dhash(Image.open(path_image))

    if hash not in hash_dict:
      hash_dict[hash] = path_image

    else:
      #print(f'{hash_dict[hash]} is equal to {path_image}')
      os.remove(path_image)
      #print(f'{path_image} removed')


# **VII. Data Description**


In [ ]:
import os

path_fire = drive_data_folder + 'fire'
path_start_fire = drive_data_folder + 'start_fire'
path_no_fire = drive_data_folder + 'no_fire'

print(f'Fire : {len(os.listdir(path_fire))} images')
print(f'Start fire : {len(os.listdir(path_start_fire))} images')
print(f'No fire : {len(os.listdir(path_no_fire))} images')